### Introduction
The `Route` component renders a component based on the page path. A simplistic implementation looks like:

In [ ]:
function Route(props) {
  const pathname = window.location.pathname;
  if (pathname.match(props.path)) {
    return createElement(props.component);
  } else {
    return null;
  }
}

// Or
function Route({ path, component: Component }) {
  const pathname = window.location.pathname;
  if (pathname.match(props.path)) {
    return <Component />;
  } else {
    return null;
  }
}

The `match` function expects a regular expression as argument and returns an array containing matches. If a string is passed it is implicitly converted to a regular expression.

If we have the following code:

In [ ]:
export default function App() {
  return (
    <div className="App">
      <h1>Some Links</h1>
      <ul>
        <li><a href="/valorant">Valorant</a></li>
        <li><a href="/counterstrike">Counter Strike</a></li>
      </ul>
      <br />
      <Route path="/valorant" component={Valorant} />
      <Route path="/counterstrike" component={CounterStrike} />
    </div>
  );
}

function Valorant(props) {
  return (
    <div>
      <h2>Valorant</h2>
      <p>By Riot</p>
    </div>
  );
}

function CounterStrike(props) {
  return (
    <div className="description">
      <h2>Counter Strike</h2>
      <p>By Valve</p>
    </div>
  );
}

The image below represents the page when a link is clicked:  
![Route Link](images/router1.png)  

The problem with using the `a` tag is that the page loads whenever we click it creating a jarring effect. So we will create our own component which would simulate behaviour of a tag. We also need to make use of the history API. React Router provides Link component whose simplistic implementation is below:

In [ ]:
function Link(props) {
  function onClick(event) {
    event.preventDefault();
    window.history.pushState(null, null, props.to);
  }

  return (
    <a href={props.to} onClick={onClick}>
      {props.children}
    </a>
  );
}

Replacing a tag with Link component is not enough. We need to make our Route components aware about the page path changes. We can utilize the history.js library for this purpose:

In [ ]:
import createHistory from 'history/createBrowserHistory';
const history = createHistory();

// Creating a force update hook
function useForceUpdate(){
    const [value, setValue] = useState(0); // integer state
    return () => setValue(value => value + 1); // update the state to force render
}

// When App re-renders, Router components will also re-render
export default function App() {
    const forceUpdate = useForceUpdate();
    
    useEffect(() => {
        history.listen(() => {
            forceUpdate();
        })
    })
    
    // ...

Instead of modifying our App component, we can create a `Router` component and put our App component inside Router. The Router component would be responsible for listening to history changes. We would also pass history and window.location packed as context such that it is available to all the child components:

In [ ]:
const history = createHistory();
const HistoryContext = createContext();

export function Router(props) {
  const forceUpdate = useForceUpdate();

  useEffect(() => {
    history.listen(() => {
      forceUpdate();
    });
  });

  const historyContext = {
    history: history,
    location: window.location
  };

  return (
    <HistoryContext.Provider value={historyContext}>
      {props.children}
    </HistoryContext.Provider>
  );
}


The Route and Link components will now use the historyContext:

In [ ]:
function Route(props) {
  const historyContext = useContext(HistoryContext);

  const pathname = historyContext.location.pathname;
  if (pathname.match(props.path)) {
    return createElement(props.component);
  } else {
    return null;
  }
}

function Link(props) {
  const historyContext = useContext(HistoryContext);

  function onClick(event) {
    event.preventDefault();
    historyContext.history.push(props.to);
  }

  return (
    <a href={props.to} onClick={onClick}>
      {props.children}
    </a>
  );
}

Then we need to create a `Redirect` component which would immediately modify location when rendered. 

In [ ]:
function Redirect(props){
    const historyContext = useContext(HistoryContext);
    
    useEffect(() => {
        historyContext.history.push(props.to);
    })
    
    return null;
}

### React Router
Is one of the most commonly used React routing library. Some of the React Router components are:

**BrowserRouter:** React provides different flavours of Router. BrowserRouter is browser specific router which uses HTML5 history API

In [ ]:
// basename is used if all your urls have a prefix
// forceRefresh emulates typical browser behaviour of refreshing page on navigation
// getUserConfirmation asks for user confirmation before navigation
const app = (
    <BrowserRouter
        basename={optionalString}
        forceRefresh={optionalBool}
        getUserConfirmation={optionalFunc}
        <App />
    </BrowserRouter>
);

**Link:** React Router's version of the a tag.

In [ ]:
const aboutLink = <Link to="/about">About</Link>

The Link component internally uses an a tag. However, we can pass our own component if we want that to be rendered instead:

In [ ]:
const FancyLink = React.forwardRef((props, ref) => (
  <a ref={ref} {...props}>💅 {props.children}</a>
))

const link = <Link to="/" component={FancyLink} />

**NavLink:** special version of Link component which allows us to conditionally apply styling to the rendered element when it matches the current URL.

In [ ]:
// We can provide a class to the link if active
const l1 = <NavLink to="/about" activeClassName="active">About</NavLink>

// Or we can specify a style
const l2 = (
<NavLink
  to="/faq"
  activeStyle={{
    fontWeight: "bold",
    color: "red"
  }}
>
  FAQs
</NavLink>
)

**Route:** renders a component when path matches current URL. The important thing to remember is that Route uses regex to match path, therefore if the url is / then all Route components in the below example are rendered:

In [ ]:
<Route path="/" component={Home} />
<Route path="/about" component={About} />
<Route path="/contact" component={Contact} />

We can add an `exact` prop which when true, will only match if the path matches the `location.pathname` exactly. 

In [ ]:
<Route exact path="/one/two"> <!-- Matches only when URL is /one/two -->

If we want to maintain case sensitivity, we can add sensitive prop

In [ ]:
<Route sensitive path="/one">
  <About />
</Route>

**Switch:** If we want to render only 1 Route component even though the path matches with multiple Routes, we use the Switch component. It renders the first child Route or Redirect that matches the location.

In [ ]:
let routes = (
  <Switch>
    <Route exact path="/">
      <Home />
    </Route>
    <Route path="/about">
      <About />
    </Route>
    <Route path="/:user">
      <User />
    </Route>
    <Route>
      <NoMatch />
    </Route>
  </Switch>
);